In [1]:
import pybaseball
import pandas as pd
import numpy as np

In [2]:
pybaseball.cache.enable()

In [3]:
pitcher_data = pybaseball.pitching_stats(2025)

print(pitcher_data.head())

     IDfg  Season                Name Team  Age   W   L  WAR   ERA   G  ...  \
1   22267    2025        Tarik Skubal  DET   28  13   6  6.6  2.21  31  ...   
0   33677    2025         Paul Skenes  PIT   23  10  10  6.5  1.97  32  ...   
4   20778    2025  Cristopher Sanchez  PHI   28  13   5  6.4  2.50  32  ...   
5   27463    2025     Garrett Crochet  BOS   26  18   5  5.8  2.59  32  ...   
15  17995    2025          Logan Webb  SFG   28  15  11  5.5  3.22  34  ...   

    Pit+ FC  Stf+ FS  Loc+ FS  Pit+ FS  Stuff+  Location+  Pitching+  Stf+ FO  \
1       NaN    113.0    142.0    157.0     116        105        119      NaN   
0       NaN     90.0    125.0    123.0     107        111        116      NaN   
4       NaN      NaN      NaN      NaN     114        101        117      NaN   
5     123.0      NaN      NaN      NaN     116         99        112      NaN   
15     94.0      NaN      NaN      NaN     105        107        111      NaN   

    Loc+ FO  Pit+ FO  
1       NaN    

In [17]:
a = (pitcher_data["K%"] - 0.1) * 8 # strikeout term
b = 2.375 - (pitcher_data["BB%"] * 12) # walk term
c = 2.375 - (pitcher_data["HBP"] / pitcher_data["TBF"] * 12) # HBP term
d = 2.375 - (pitcher_data["HR"] / pitcher_data["TBF"] * 52) # home run term

print(f"a ranges from {a.min()} to {a.max()}")
print(f"b ranges from {b.min()} to {b.max()}")
print(f"c ranges from {b.min()} to {b.max()}")
print(f"d ranges from {d.min()} to {d.max()}")

a ranges from 0.33599999999999985 to 1.776
b ranges from 0.9589999999999999 to 1.871
c ranges from 0.9589999999999999 to 1.871
d ranges from -0.1358005082592122 to 1.6017657992565058


In [21]:
def clamp(
    s: pd.Series,
    min: float = 0.0,
    max: float = 2.375,
) -> pd.Series:
    """
    Set all values above the max to max, and all values below the min to the min.
    """
    s = s.apply(lambda x: min if x < min else x)
    s = s.apply(lambda x: max if x > max else x)

    return s

a = clamp(a)
b = clamp(b)
c = clamp(c)
d = clamp(d)

ratings = (a + b + c + d) / 6 * 100

print(f"ratings range from {ratings.min()} to {ratings.max()}")

ratings range from 69.57522236340533 to 118.70698044565711


In [22]:
pitcher_data["rating"] = ratings

print(pitcher_data.sort_values(by="rating", ascending=False).head())

     IDfg  Season                Name Team  Age   W   L  WAR   ERA   G  ...  \
0   33677    2025         Paul Skenes  PIT   23  10  10  6.5  1.97  32  ...   
1   22267    2025        Tarik Skubal  DET   28  13   6  6.6  2.21  31  ...   
4   20778    2025  Cristopher Sanchez  PHI   28  13   5  6.4  2.50  32  ...   
15  17995    2025          Logan Webb  SFG   28  15  11  5.5  3.22  34  ...   
5   27463    2025     Garrett Crochet  BOS   26  18   5  5.8  2.59  32  ...   

    Stf+ FS  Loc+ FS  Pit+ FS  Stuff+  Location+  Pitching+  Stf+ FO  Loc+ FO  \
0      90.0    125.0    123.0     107        111        116      NaN      NaN   
1     113.0    142.0    157.0     116        105        119      NaN      NaN   
4       NaN      NaN      NaN     114        101        117      NaN      NaN   
15      NaN      NaN      NaN     105        107        111      NaN      NaN   
5       NaN      NaN      NaN     116         99        112      NaN      NaN   

    Pit+ FO      rating  
0       NaN 

In [23]:
print(pitcher_data.sort_values(by="rating", ascending=True).head())

     IDfg  Season             Name   Team  Age   W   L  WAR   ERA   G  ...  \
51  21504    2025       Jake Irvin    WSN   28   9  13 -0.4  5.70  33  ...   
50  27636    2025  Mitchell Parker    WSN   25   9  16  0.9  5.68  33  ...   
24  15823    2025     Zack Littell  - - -   29  10   8  1.5  3.81  32  ...   
31  17677    2025  Jeffrey Springs    ATH   32  11  11  1.7  4.11  32  ...   
32  14168    2025     Jose Berrios    TOR   31   9   5  1.3  4.17  31  ...   

    Stf+ FS  Loc+ FS  Pit+ FS  Stuff+  Location+  Pitching+  Stf+ FO  Loc+ FO  \
51      NaN      NaN      NaN      94        103         96      NaN      NaN   
50     98.0    101.0     95.0      91        103         94      NaN      NaN   
24    103.0     98.0     98.0      85        104         93      NaN      NaN   
31      NaN      NaN      NaN      94         98         93      NaN      NaN   
32      NaN      NaN      NaN      89        102         93      NaN      NaN   

    Pit+ FO     rating  
51      NaN  69.575